In [ ]:
!nvidia-smi 

Setup library

In [ ]:
import json 
import pandas as pd 
import numpy as np 
import torch 
from pathlib import Path 
from torch.utils.data import Dataset , DataLoader 
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint 
from pytorch_lightning.loggers import TensorBoardLogger 
from sklearn.model_selection import train_test_split 
from termcolor import colored 
import textwrap 
from transformers import (
    AdamW , T5ForConditionalGeneration , 
    T5TokenizerFast as T5Tokenizer 
)
from tqdm.auto import tqdm 

In [ ]:
# Thiết lập một số cấu hình đồ thị tiêu chuẩn 
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16 , 10

In [ ]:
# Thiết lập hạt giống tất cả để đảm bảo rằng mọi thứ có thể tái tạo được 
pl.seed_everything(42)

Download The Dataset 

In [ ]:
!gdown --id 1YL77L4lVadhM4ppu5p4hKiLVR4l5soo3

In [ ]:
!unzip -q news_summary.csv.zip -d data

In [ ]:
df = pd.read_csv("data/news_summary.csv", encoding="latin-1")
df.head()

In [ ]:
df = df[['text' , "ctext"]]
df.head()

In [ ]:
df.columns = ["summary","text"]
df = df.dropna()
df.head()

In [ ]:
df.shape

In [ ]:
train_df , test_df = train_test_split(df, test_size=0.15)
train_df.shape , test_df.shape 

In [ ]:
# Tạo một tập dữ liệu ở dạng tập ngày mở rộng 
# Từ tập dữ liệu pytorch 
class NewsSummaryDataset(Dataset):
    def __init__(
            self, 
            data: pd.DataFrame, 
            tokenizer: T5Tokenizer, 
            text_max_token_len = 512 , 
            summary_max_token_len: int = 128 
    ):
        self.tokenizer = tokenizer 
        self.data = data 
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len 
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        # Lấy ra hàng dữ  liệu và chỉ mục hiện tại 
        data_row = self.iloc[index]
        # Trích suất văn bản theo chỉ mục 
        text = data_row["text"]
        # Thực hiện mã hóa văn bản 
        text_encoding = self.tokenizer(
            text , max_length= self.text_max_token_len , 
            padding ="max_length",
            truncation =True , 
            return_attention_mask=True , 
            # Thêm vào các mã thông báo đặc biệt 
            add_special_tokens=True , 
            # Trả về dạng  tensor 
            return_tensor="pt"
        )
        # Mã hóa văn bản tóm tắt 
        summary_encoding = self.tokenizer (
            data_row["summary"], max_length= self.summary_max_token_len, 
            padding ="max_length", 
            truncation = True , 
            return_attention_mask= True , 
            # Thêm vào các mã thông báo đặc biệt 
            add_special_tokens= True , 
            # Trả vềd ạng tensor 
            return_tensors="pt",
        )

        # Tạo ra các nhãn thực tế 
        labels = summary_encoding["input_ids"]
        labels [labels == 0] = -100

        # Trả về 1 danh sách các bộ tham số dưới dạng từ điển
        return dict(
            # DATA = TEXT VÀ SUMMARY 
            text = text , summary = data_row["summary"], 
            # input_ids 
            text_input_ids = text_encoding["input_ids"].flatten(),
            # attention_mask 
            text_attention_mask = text_encoding["attention_mask"].flatten(),
            # labels 
            labels = labels.flatten(),
            # labels attention mask summary 
            labels_attention_mask = summary_encoding["attention_mask"].flatten()
        )


In [ ]:
# Xây dựng lớp trả về dữ liệu 
class NewsSummaryDataModule(pl.LightningDataModule):
    # Thiết lập phương thức khởi tạo
    def __init__(self,
        train_df: pd.DataFrame, 
        test_df: pd.DataFrame , 
        tokenizer: T5Tokenizer , 
        batch_size :int = 8, 
        text_max_token_len: int = 512 , 
        summary_max_token_len: int = 128 ,
        ):
        super().__init__()
        self.train_df = train_df 
        self.test_df = test_df 
        self.batch_size = batch_size 
        self.tokenizer = tokenizer 
        self.text_max_token_len = text_max_token_len 
        self.summary_max_token_len = summary_max_token_len
    # Xây dựng phương thức thiết lập 
    # mã hóa data Train và Test 
    def setup(self, stage=None):
        self.train_df = NewsSummaryDataset(
            self.train_df , self.tokenizer , 
            self.text_max_token_len , 
            self.summary_max_token_len , 
        )

        self.test_dataset = NewsSummaryDataset(
            self.test_df ,
            self.tokenizer ,
            self.text_max_token_len ,
            self.summary_max_token_len,
        )
    # Xây dựng phương thức thực hiện mô hình với dữ liệu 
    def train_datalogger(self):
        return DataLoader(
            self.train_dataset ,
            batch_size = self.batch_suze ,
            shuffle = True ,
            num_workers=2
        )

    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size = self.batch_size ,
            shuffle = True ,
            num_workers=2
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size = self.batch_size ,
            shuffle = True ,
            num_workers=2
        )

In [ ]:
# Đào tạo trước với T5 base
MODEL_NAME = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [ ]:
# Tạo 2 danh sách rỗng để chứa số lượng 
# len(tokenizer.encode(row['text']))
# len(tokenizer.encode(row['summary'])) 
text_token_counts , summary_token_counts = [] , []
for _, row in train_df.iterrows():
    text_token_count = len(tokenizer.encode(row['text']))
    text_token_counts.append(text_token_count)

    summary_token_count = len(tokenizer.encode(row['summary']))
    summary_token_counts.append(summary_token_count)

In [ ]:
# Vẽ biểu đồ histplot cho danh sách token_count và summary_token_counts 
fig , (ax1 , ax2) = plt.subplots(1,2)

sns.histplot(text_token_counts , ax=ax1)
ax1.set_title("full text token counts")

sns.histplot(summary_token_counts , ax=ax2)
ax2.set_title("full text token counts")

In [ ]:
N_EPOCHS = 3
BATCH_SIZE = 8
data_module = NewsSummaryDataModule(train_df , test_df , tokenizer , batch_size=BATCH_SIZE)

In [ ]:
class NewsSummaryModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME , return_dict=True)
        self.tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

    def forward(self, input_ids , attention_mask , decoder_attention_mask , labels=None):
        decoder_input_ids = self.tokenizer(labels , return_tensor="pt").input_ids 
        output = self.model(
            input_ids , 
            attention_masl = attention_mask , 
            decoder_attention_mask = decoder_attention_mask , 
            decoder_input_ids = decoder_input_ids ,
        )
        return output.loss , output.logits 
    
    def training_step(self, batch , batch_idx):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss , outputs = self(
            input_ids = input_ids ,
            attention_mask = attention_mask ,
            decoder_attention_mask = labels_attention_mask ,
            labels = labels

        )

        self.log("train_loss", loss , prog_bar=True ,logger=True )
        return loss


    def validation_step(self, batch , batch_idx):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss , outputs = self(
            input_ids = input_ids ,
            attention_mask = attention_mask ,
            decoder_attention_mask = labels_attention_mask ,
            labels = labels

        )

        self.log("val_loss", loss , prog_bar=True ,logger=True )
        return loss

    def test_step(self, batch , batch_idx):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss , outputs = self(
            input_ids = input_ids ,
            attention_mask = attention_mask ,
            decoder_attention_mask = labels_attention_mask ,
            labels = labels

        )

        self.log("test_loss", loss , prog_bar=True ,logger=True )
        return loss


    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.001)




In [ ]:
model = NewsSummaryModel()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath = "checkpoints",
    filename = "best-checkpoint",
    save_top_k= 1 , 
    verbose =True , 
    monitor="val_los", 
    mode="min"
)
logger = TensorBoardLogger("lightning_logs", name="news-summary")

In [ ]:
trainer = pl.Trainer(logger=logger , callbacks=[checkpoint_callback],
                    devieces=1)

In [ ]:
trainer.fit(model , data_module)

In [ ]:
trained_model = NewsSummaryModel.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path 
)
trained_model.freeze()

In [ ]:
def summarize(text):
    text_encoding =tokenizer(
        text , max_length = 512 , 
        padding="max_length",
        truncation =True , return_attention_mask =True , 
        add_special_tokens=True , 
        return_tensor="pt"
    ) 
    generated_ids = trained_model.model.generate(
        input_ids =text_encoding["input_ids"],
        attention_mask = text_encoding["attention_mask"],
        max_length = 150 ,
        num_beams=2 ,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )
    preds= [
      tokenizer.decode(gen_id , skip_special_tokens=True ,
                       clean_up_tokenization_spaces=True )
      for gen_id in generated_ids
  ]
    return "".join(preds)

In [ ]:
sample_row = test_df.iloc[0]
text = sample_row["text"]
model_summary = summarize(text)

In [ ]:
text 

In [ ]:
sample_row["summary"]

In [ ]:
import tensorflow as tf 
from tensorflow import keras

In [ ]:
import keras_nlp 

In [ ]:
import tensorflow_datasets 